In [5]:
import pandas as pd, os

In [6]:
fileNames=[fileName for fileName in os.listdir("data/data_raw") if (os.path.isfile(f"data/data_raw/{fileName}") and 
                                                                    not os.path.exists(f"data/data_cleaned/{fileName}"))]

In [7]:
def  cleanFile(fileName, outFileName):
    df = pd.read_csv(file_name)
    df=df.drop_duplicates()
    df=df[df["Taken"]!="Taken"]
    df=df[df["Dept_Date"].map(len)>10]  #removing entries that only had date and no time
    dept_date_time=pd.to_datetime(df["Dept_Date"], format="%Y-%m-%dT%H:%M:%SZ") #parsing date to pandas.date
    df.insert(2, "Dept_Date_Time", dept_date_time)
    df["Dept_Date"]=df["Dept_Date_Time"].dt.date
#   df["Dept_Time"]=pd.to_datetime(df["Dept_Time"], format="%H:%M") #Don't Need this 
#   df.drop(columns="Dept_Time", inplace=True)
    df.drop(columns="Available", inplace=True)
    df.drop(columns="Hopping", inplace=True)
    df["Duration"]=pd.to_timedelta(df["Duration"])
    df["Arr_Time"]=df["Dept_Date_Time"]+df["Duration"]
    df=df[df["Duration"]<pd.to_timedelta("03:00:00")]
    dept_weekday=df["Dept_Date_Time"].apply(lambda x:x.weekday_name)
    df.insert(5, "Dept_Weekday", dept_weekday) #adding dept_weekday at column 4
    
    taken_date_time=pd.to_datetime(df["Taken"], format="%Y-%m-%d")
    df['Taken_DT'] = taken_date_time
    taken_weekday=df["Taken_DT"].apply(lambda x:x.weekday_name)
    df['Taken_Day'] = taken_weekday
    
   # dura=pd.to_timedelta(df['Duration'])
   # df['Duration_N'] = dura
    
    airlines_to_drop=["AirAsia","Go Air", "Vistara", "Spicejet HBag Only"]
    for airline in airlines_to_drop:
        df=df[df["Airways"]!=airline]
    # Write the results to a different file
    df.to_csv(file_name_output, index=False)
    return None

In [8]:
for fileName in fileNames:
    file_name = f"data/data_raw/{fileName}"
    file_name_output = f"data/data_cleaned/{fileName}"
    cleanFile(file_name, file_name_output)